# ISB Research Associate Test

This Project is my attempt to complete the following tasks.

1. Real Returns
2. Panel Data
3. Polled OLS
4. Fixed Effects
5. Traditional Risk-Return Pardign
6. Failure of CAPM
7. Fama-French 3 factor Model
8. Price of Risk / Factor
9. 2008 Performance?

### Import the required modules

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels import regression

from linearmodels.panel import PooledOLS
from linearmodels.panel import PanelOLS

import scipy
from matplotlib import pyplot as plt
%matplotlib inline

#### Importing Data

In [2]:
data = pd.read_excel('industry_data.xlsx')
data.tail()

,year,Agric,Food,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Insur,RlEst,Fin,Other,excess market returns,SMB,HML,risk free rate,real personal consumption,price index
85,2013,20.55,41.53,62.45,22.81,63.51,49.85,63.74,31.59,40.16,...,48.63,46.39,48.00,27.10,35.20,7.27,1.35,0.02,10868.396,236.736167
86,2014,-8.63,16.90,40.66,28.04,12.88,-1.46,21.09,4.82,-0.57,...,12.62,2.86,9.71,-5.59,11.70,-8.00,-1.58,0.02,11264.268,237.017000
87,2015,-24.25,5.30,2.51,27.51,-14.88,-1.56,-4.17,-8.53,-18.83,...,3.33,-11.28,-13.01,-8.87,0.07,-3.94,-9.56,0.02,11572.124,240.007167
88,2016,18.76,18.76,18.19,19.84,11.91,23.62,23.32,24.98,6.29,...,27.32,17.60,18.58,20.98,13.30,6.60,22.91,0.20,11890.700,245.119583
89,2017,7.67,6.26,28.88,32.26,-5.64,34.56,5.12,2.65,29.81,...,12.83,21.47,21.77,-1.91,NaN,NaN,NaN,NaN,NaN,NaN


## 1. Real returns
### Feature engineering and Dealing with missing Data

In [3]:
#remove last row(2017)
data = data.iloc[:-1]

In [4]:
# Calculate inflation and Market Returns
data['Inflation'] = data['price index'].pct_change()*100
data = data.fillna(round(data.mean(),2))
data['Market Returns'] = data['excess market returns'] + data['risk free rate']

In [5]:
data = data.set_index('year')
data.head()

,Agric,Food,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,Hlth,...,Fin,Other,excess market returns,SMB,HML,risk free rate,real personal consumption,price index,Inflation,Market Returns
year,,,,,,,,,,,,,,,,,,,,,
1928,-3.10,31.87,41.05,26.69,6.60,63.73,25.84,34.45,5.19,18.36,...,37.34,12.18,35.39,4.20,-6.15,3.56,780.977,17.158333,3.140000,38.95
1929,-14.30,-31.88,-28.02,-43.55,-73.96,-51.78,-16.99,-39.33,-38.93,18.36,...,-1.00,-46.91,-19.54,-30.80,11.81,4.75,739.142,16.700000,-2.671200,-14.79
1930,-49.86,-25.81,-57.29,-22.07,-25.63,-16.26,-13.32,-50.74,-34.78,18.36,...,-50.56,-18.72,-31.23,-5.13,-12.28,2.41,716.055,15.208333,-8.932136,-28.82
1931,-39.01,-32.43,-23.01,-9.71,-66.35,-76.94,-57.72,-45.37,-42.81,18.36,...,-49.08,-17.04,-45.11,3.53,-14.29,1.07,651.949,13.641667,-10.301370,-44.04
1932,114.86,9.97,58.40,-4.46,-33.34,-0.37,-25.58,7.56,75.86,18.36,...,41.11,-18.23,-9.39,4.67,10.15,0.96,637.569,12.933333,-5.192425,-8.43


In [6]:
data = data[['Agric', 'Food ', 'Beer ', 'Smoke', 'Toys ', 'Fun  ', 'Books', 'Hshld',
       'Clths', 'Hlth ', 'MedEq', 'Drugs', 'Chems', 'Rubbr', 'Txtls', 'BldMt',
       'Cnstr', 'Steel', 'Mach ', 'ElcEq', 'Autos', 'Aero ', 'Ships', 'Guns ',
       'Gold ', 'Mines', 'Coal ', 'Oil  ', 'Util ', 'Telcm', 'PerSv', 'BusSv',
       'Comps', 'Chips', 'LabEq', 'Paper', 'Boxes', 'Trans', 'Whlsl', 'Rtail',
       'Meals', 'Banks', 'Insur', 'RlEst', 'Fin  ', 'Other',
       'excess market returns', 'SMB', 'HML', 'risk free rate','Market Returns',
       'real personal consumption', 'price index', 'Inflation']]

In [7]:
Returns = data[data.columns[0:50]]
Inflation = data[data.columns[53]]

In [8]:
for column in Returns.columns:
    Returns[column] = Returns[column] - Inflation
Returns.head()    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Agric,Food,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,Hlth,...,Meals,Banks,Insur,RlEst,Fin,Other,excess market returns,SMB,HML,risk free rate
year,,,,,,,,,,,,,,,,,,,,,
1928,-6.240000,28.730000,37.910000,23.550000,3.460000,60.590000,22.700000,31.310000,2.050000,15.220000,...,23.950000,121.120000,9.460000,61.480000,34.200000,9.040000,32.250000,1.060000,-9.290000,0.420000
1929,-11.628800,-29.208800,-25.348800,-40.878800,-71.288800,-49.108800,-14.318800,-36.658800,-36.258800,21.031200,...,1.241200,-28.448800,-38.708800,14.581200,1.671200,-44.238800,-16.868800,-28.128800,14.481200,7.421200
1930,-40.927864,-16.877864,-48.357864,-13.137864,-16.697864,-7.327864,-4.387864,-41.807864,-25.847864,27.292136,...,-10.417864,-6.687864,-26.967864,-41.467864,-41.627864,-9.787864,-22.297864,3.802136,-3.347864,11.342136
1931,-28.708630,-22.128630,-12.708630,0.591370,-56.048630,-66.638630,-47.418630,-35.068630,-32.508630,28.661370,...,-31.528630,-28.918630,-43.208630,-54.728630,-38.778630,-6.738630,-34.808630,13.831370,-3.988630,11.371370
1932,120.052425,15.162425,63.592425,0.732425,-28.147575,4.822425,-20.387575,12.752425,81.052425,23.552425,...,-15.207575,-9.867575,-21.307575,-33.297575,46.302425,-13.037575,-4.197575,9.862425,15.342425,6.152425


In [9]:
Returns.describe()

,Agric,Food,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,Hlth,...,Meals,Banks,Insur,RlEst,Fin,Other,excess market returns,SMB,HML,risk free rate
count,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,...,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000
mean,10.748636,12.519535,14.623804,16.424928,10.732456,14.861669,14.727624,12.196838,13.160096,15.218299,...,12.722344,16.548074,11.229085,13.747624,14.387063,11.421220,4.983355,0.255490,2.008186,0.293467
std,33.234016,25.812788,39.863052,30.670240,39.729397,39.964683,40.467947,32.019239,37.415965,27.359349,...,38.272477,36.360265,25.280749,47.924954,32.314764,34.958085,20.378334,13.968865,14.186672,3.949912
min,-48.922767,-36.302190,-48.357864,-40.878800,-71.288800,-66.638630,-69.614454,-54.034805,-64.284805,-65.464805,...,-64.084805,-43.914964,-47.834805,-66.244805,-52.914805,-60.524805,-44.893147,-34.514805,-35.536857,-14.039411
25%,-11.589337,-3.800095,-7.755054,-1.646002,-13.659463,-10.520111,-8.691508,-8.785644,-11.956002,7.437533,...,-10.417864,-4.766857,-3.885962,-18.184563,-5.550095,-8.159100,-9.595644,-8.118627,-8.209294,-1.540043
50%,5.052558,10.733829,10.807722,13.340840,8.102997,13.845167,14.088573,10.024946,12.159036,16.086196,...,7.990798,10.431373,9.460000,6.848667,12.958573,8.173930,7.327253,-1.038773,2.178796,0.558343
75%,27.361088,24.004946,26.485187,29.803854,32.342763,39.861005,28.605167,31.310000,30.588316,21.811952,...,31.482558,36.283968,26.777721,38.017253,35.655187,25.649036,18.034946,8.160798,12.149316,1.955546
max,121.830619,134.770619,265.840619,129.900619,118.128204,157.358204,197.029957,101.660619,177.520619,109.541180,...,184.328204,203.510619,86.887073,211.533024,113.413024,203.610619,53.570619,46.968204,37.333024,11.371370


## 2. Panel data

In [10]:
df1 = Returns[Returns.columns[:46]]
df1 = df1.stack()
df1 = df1.to_frame()

df2 = Returns[Returns.columns[46]]
df2 = df2.to_frame()

Panel = df1.join(df2)
Panel.index.rename('Industry',level=1,inplace=True)
Panel = Panel.swaplevel(i=-2, j=-1, axis=0)
Panel.columns = ['Returns','Market']
Panel.head()

,,Returns,Market
Industry,year,,
Agric,1928,-6.24,32.25
Food,1928,28.73,32.25
Beer,1928,37.91,32.25
Smoke,1928,23.55,32.25
Toys,1928,3.46,32.25


## 3. Pooled OLS

In [11]:
mod = PooledOLS.from_formula('Returns ~ 1 + Market', data=Panel)
print(mod.fit())

                          PooledOLS Estimation Summary                          
Dep. Variable:                Returns   R-squared:                        0.4326
Estimator:                  PooledOLS   R-squared (Between):              0.0000
No. Observations:                4094   R-squared (Within):               0.4338
Date:                Thu, Jul 11 2019   R-squared (Overall):              0.4326
Time:                        08:59:24   Log-likelihood                 -1.94e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3120.2
Entities:                          46   P-value                           0.0000
Avg Obs:                       89.000   Distribution:                  F(1,4092)
Min Obs:                       89.000                                           
Max Obs:                       89.000   F-statistic (robust):             3120.2
                            

* Intercept($\alpha$) is the expected mean of the dependent variable
* coefficient($\beta$) tells us how much dependent variable will change if we change independent variable

## 4. Fixed effects regression

In [12]:

mod1 = PanelOLS(Panel.Returns, sm.add_constant(Panel.Market), entity_effects=True)
fe_res = mod1.fit()
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                Returns   R-squared:                        0.4338
Estimator:                   PanelOLS   R-squared (Between):              0.0000
No. Observations:                4094   R-squared (Within):               0.4338
Date:                Thu, Jul 11 2019   R-squared (Overall):              0.4326
Time:                        08:59:27   Log-likelihood                -1.939e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3101.0
Entities:                          46   P-value                           0.0000
Avg Obs:                       89.000   Distribution:                  F(1,4047)
Min Obs:                       89.000                                           
Max Obs:                       89.000   F-statistic (robust):             3101.0
                            

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### Why do you think new α and β have changed? How do you interpret?


α and β have changed here because Fixed effects is essentially regressing the time demeaned data. Which here is the industry excess return for each year.

## 5. Traditional Risk-Return Paradign

In [13]:
stats = data[data.columns[:46]].describe()
Mean = stats.iloc[1]
STD = stats.iloc[2]

In [14]:
X = sm.add_constant(STD)
model = regression.linear_model.OLS(Mean, X).fit()
a = model.params[0]
b = model.params[1]


Y_hat = STD * b + a

In [15]:
import plotly.plotly as py
import plotly.graph_objs as go



# Create a trace
trace = go.Scatter(
    x = STD,
    y = Mean,
    name='Risk-Return',
    mode = 'markers'
)

trace2 = go.Scatter(
    x = STD,
    y = Y_hat,
    name = 'Prediction',
    mode = 'lines'
)

layout = go.Layout(xaxis=dict(title='STD'),yaxis=dict(title='Mean'),title=' risk-return paradigm')

dat1 = [trace, trace2]

fig= go.Figure(data=dat1, layout=layout)
py.iplot(fig)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



###  Does naive risk-return paradigm seems to hold?

Yes, it does seem to hold from the plot above

## 6. Failure of CAPM?

In [16]:
assets = Panel.index.levels[0].unique()
Inds =  [Panel.loc[asset].Returns for asset in assets]

In [17]:
Facts = data[data.columns[46:49]]

In [18]:
R_F = data['risk free rate']

In [19]:
M = Facts['excess market returns']

In [20]:
CAPM_Beta = [regression.linear_model.OLS(R-R_F, sm.add_constant(M)).fit().params[1] for R in Inds]
CAPM_Beta = pd.DataFrame(CAPM_Beta)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning:

Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.



In [21]:
CAPM_Beta.columns = ['Market']

In [22]:
trace3 = go.Scatter(
    x = CAPM_Beta.Market,
    y = Mean,
    mode = 'markers',
    text = assets,
    name='Returns'
)

trace4 = go.Scatter(
    y = regression.linear_model.OLS(Mean, sm.add_constant(list(CAPM_Beta.Market))).fit().predict(),
    x = CAPM_Beta.Market,
    mode = 'lines',
    text = assets,
    name='Prediction'
)


dat4 = [trace3,trace4]

# Plot and embed in ipython notebook!
py.iplot(dat4, filename='basic-scatter')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



CAPM fails to hold the industry level.

CAPM fails because it says that the expected return of a security or a portfolio equals the risk-free rate plus a risk premium for taking systematic risk. But it fails empirical tests. It is  one factor model with market return as the independent variable

## 7. Fama-French 3 factor Model

In [23]:
ThreeFactor_betas = [regression.linear_model.OLS(R-R_F, Facts).fit().params for R in Inds]

In [24]:
SMB = []
for i in range(len(ThreeFactor_betas)):
    SMB.append(ThreeFactor_betas[i]['SMB'])
    

In [25]:
HML = []
for i in range(len(ThreeFactor_betas)):
    HML.append(ThreeFactor_betas[i]['HML'])

In [26]:
New_Betas = pd.DataFrame(ThreeFactor_betas)

In [27]:
trace5 = go.Scatter(
    x = SMB,
    y = Mean,
    mode = 'markers'
)


dat5 = [trace5]

# Plot and embed in ipython notebook!
py.iplot(dat5, filename='basic-scatter')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [28]:
# Create a trace
trace6 = go.Scatter(
    x = HML,
    y = Mean,
    mode = 'markers'
)


dat6 = [trace6]

# Plot and embed in ipython notebook!
py.iplot(dat6, filename='basic-scatter')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## CAPM Beta and Three-Factor model Beta comparision

In [30]:
trace11 = go.Bar(
    y=New_Betas['excess market returns'],
    x=assets,
    name='Three-Factor model Beta',
    marker=dict(
        color='rgb(55, 83, 109)'
    )
)
trace22 = go.Bar(
    y=CAPM_Beta.Market,
    x=assets,
    name='CAPM Beta',
    marker=dict(
        color='rgb(26, 118, 255)'
    )
)

layout1 = go.Layout(xaxis=dict(title='Industry'),yaxis=dict(title='Beta'),title='CAPM VS 3-Factor')

data33 = [trace11, trace22]

fig = go.Figure(data=data33,layout=layout1)
py.iplot(fig)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## 8. Price of Risk/ Factor

In [31]:
Mean.index = New_Betas.index

In [32]:
fam = regression.linear_model.OLS(Mean, sm.add_constant(New_Betas)).fit()

In [33]:
fam.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mean   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     1.446
Date:                Thu, 11 Jul 2019   Prob (F-statistic):              0.243
Time:                        09:00:23   Log-Likelihood:                -93.284
No. Observations:                  46   AIC:                             194.6
Df Residuals:                      42   BIC:                             201.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    15.6796      1.335     11.746      0.000      12.986      18.374
excess market returns     1.5164      1.254      1.209      0.233      -1.014       4.047
SMB                       0.4827      0.793      0.609      0.546      -1.118       2.083
HML                      -1.0993      0.946     -1.162      0.252      -3.009       0.811
==============================================================================
Omnibus:                        2.238   Durbin-Watson:                   2.270
Prob(Omnibus):                  0.327   Jarque-Bera (JB):                1.350
Skew:                           0.374   Prob(JB):                        0.509
Kurtosis:                       3.379   Cond. No.                         10.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
fam2 = regression.linear_model.OLS(Mean, sm.add_constant(CAPM_Beta)).fit()
fam2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mean   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.4079
Date:                Thu, 11 Jul 2019   Prob (F-statistic):              0.526
Time:                        09:00:24   Log-Likelihood:                -95.333
No. Observations:                  46   AIC:                             194.7
Df Residuals:                      44   BIC:                             198.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.6648      1.186     14.053      0.000      14.275      19.055
Market         0.6016      0.942      0.639      0.526      -1.297       2.500
==============================================================================
Omnibus:                        1.790   Durbin-Watson:                   2.264
Prob(Omnibus):                  0.409   Jarque-Bera (JB):                1.238
Skew:                           0.400   Prob(JB):                        0.539
Kurtosis:                       3.085   Cond. No.                         8.28
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Which factors priced in equities?

1. Book to price
2. P/E ratio
3. Market cap
4. Momentum
5. Operating cashflow
6. 

## 9. 2008 Performance?

In [35]:
Crash = df1.loc[2008]

In [36]:
Crash.columns = ['Returns']

In [37]:
Crash['Betas'] = list(CAPM_Beta.Market)

In [38]:
Crash.sort_values(by=['Returns']).head()

,Returns,Betas
Industry,,
Books,-69.614454,1.312064
Autos,-64.484454,1.575572
RlEst,-62.784454,1.630646
Paper,-60.494454,1.291350
Mines,-59.894454,1.427566


In [40]:
trace8 = go.Scatter(
    x = Crash.Betas,
    y = Crash.Returns,
    text = Crash.index,
    mode = 'markers'
)

layout2 = go.Layout(xaxis=dict(title='Beta'),yaxis=dict(title='Returns'),title='Beta VS Returns in 2008')

dat8 = [trace8]

# Plot and embed in ipython notebook!
fig = go.Figure(data=dat8,layout=layout2)
py.iplot(fig)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



### No significant evidence that industries with high market β sank during 2008